In [ ]:
import sys
sys.path.append('..')
from moment_invariant_tools.construct_terms import iter_tensor_terms
from moment_invariant_tools.search_invariants import perform_search, test_independence
from moment_invariant_tools.graph_check import graph_from_contraction, draw_graph
import networkx
import matplotlib
import matplotlib.pyplot as plt

PLOT_DIRECTORY='./paper_figures'

In [ ]:
import networkx as nx
import collections
from matplotlib.patches import Arc


def pad_contraction(contraction, total_rank, dummy_start=0):
    padded = []
    max_index = max(max(indices) for indices in contraction)
    dummy = max_index + 1
    for tensor in contraction:
        extra_pairs = (total_rank - len(tensor)) // 2
        new_tensor = list(tensor)
        for _ in range(extra_pairs):
            new_tensor.extend([dummy, dummy])
            dummy += 1
        padded.append(tuple(new_tensor))
    return tuple(padded)

input_contraction = ((0, 1, 2), (0, 1, 2))
result = pad_contraction(input_contraction, 5)
print(result)  # Expected: ((0, 1, 2, 20, 20), (0, 1, 2, 21, 21))

def pad_mixed_contraction(contraction, rank_map, dummy_start=0):
    padded = []
    max_index = max(max(indices) for indices in contraction)
    dummy = max_index+1
    for tensor in contraction:
        orig_rank = len(tensor)

        # Get the target rank for this tensor's original rank.
        if orig_rank not in rank_map:
            raise ValueError(f"No target rank defined for tensor of rank {orig_rank}")
        target_rank = rank_map[orig_rank]
        extra = target_rank - orig_rank
        # We add dummy indices in pairs.
        if extra % 2 != 0:
            raise ValueError("Extra slots must be an even number to pad with pairs.")
        extra_pairs = extra // 2
        new_tensor = list(tensor)
        for _ in range(extra_pairs):
            new_tensor.extend([dummy, dummy])
            dummy += 1
        padded.append(tuple(new_tensor))
    return tuple(padded)

input_contraction = ((0,), (1,), (0, 2), (1, 2))
rank_map = {1: 5, 2: 4}
print(pad_mixed_contraction(input_contraction, rank_map))

def is_mixed(contraction):
    # A contraction is "mixed" if not all tensors have the same length (i.e. rank)
    ranks = {len(tensor) for tensor in contraction}
    return len(ranks) > 1
    
contractions = [((1, 2, 3), (1, 2, 4), (3, 4, 5), (5,0,0,0,0))]
for c in contractions:
    print("Drawing:",c)
    g = graph_from_contraction(c)
    fig,ax = plt.subplots(1,1,figsize=(3,3))
    draw_graph(g)
    # plt.show()
    plt.savefig(f'{PLOT_DIRECTORY}/pure{0}_{0}.png')
    plt.show()
    plt.close()

g = networkx.Graph()
g.add_nodes_from(range(1))
fig,ax = plt.subplots(1,1,figsize=(4,4))
pos = nx.spring_layout(g, iterations=10000)
nx.draw_networkx(g, pos)


In [ ]:
# draw the pure invariants
for i in range(1,7):
    grads = perform_search({i},10,one_invariant_per_term=False)
    contractions = [x[1] for x in grads.keys()]

    for k in range(i, 7, +2):
        print(i,k)
        index = 0
        for c0 in contractions:
            c = pad_contraction(c0, k)
            print(c)

            g = graph_from_contraction(c)
            fig,ax = plt.subplots(1,1,figsize=(3,3))
            draw_graph(g)
            # plt.show()
            plt.savefig(f'{PLOT_DIRECTORY}/pure^{k}_{i}({index}).png') 
            plt.show()
            plt.close()
            index = index + 1

In [ ]:
# draw the zero invariants
for k in range(0, 7, +2): 
    g = networkx.Graph()
    g.add_node(0)
    fig,ax = plt.subplots(1,1,figsize=(3,3))
    pos = nx.spring_layout(g)
    nx.draw_networkx_nodes(g, pos, node_color='black')
    # Draw Labels
    nx.draw_networkx_labels(g, pos, labels={0: k}, font_color='white', font_weight='bold')
    # Equal Aspect Ratio
    plt.axis('equal')
    plt.savefig(f'{PLOT_DIRECTORY}/pure^{k}_{0}({0}).png')  
    plt.show()
    plt.close()

In [ ]:
# draw the mixed hom. invariants
for i in range(1,7):
    for j in range(i+1,7):
        grads = perform_search({i,j},10,one_invariant_per_term=False)
        contractions = [x[1] for x in grads.keys()]
        print(contractions)

        for k in range(i, 7, +2):
            for l in range(j, 7, +2):
                print(k,i,l,j)
                index = 0
                for c0 in contractions:
                    if is_mixed(c0):
                        rank_map = {i: k, j: l}
                        c = pad_mixed_contraction(c0, rank_map)
                        print(c)
            
                        g = graph_from_contraction(c)
                        fig,ax = plt.subplots(1,1,figsize=(3,3))
                        draw_graph(g)
                        # plt.show()
                        plt.savefig(f'{PLOT_DIRECTORY}/mixed^{k}_{i}^{l}_{j}({index}).png')  
                        plt.show()
                        plt.close()
                        index = index + 1


In [ ]:
# now we test the mixed sim. invariants

# 13_1: 1
# 15_1: 1
# 3_15_1: 1

# 24_2: 3
# 35_3: 3
# 26_2: 3
# 46_4: 3
# 4_26_2: 3

pure = [[] for _ in range(7)]
for i in range(1,7):
    grads = perform_search({i},10,one_invariant_per_term=False)
    pure[i] = [x[1] for x in grads.keys()]

In [ ]:
contractionsPureLower = [[] for _ in range(7)]
contractionsPureLower[2] = [((0,0),),]
contractionsPureLower[3] = [pad_contraction(c, 3) for c in pure[1]]
contractionsPureLower[4] = [pad_contraction(c, 4) for c in pure[2]] + [pad_contraction(c, 4) for c in contractionsPureLower[2]]
contractionsPureLower[5] = [pad_contraction(c, 5) for c in pure[1]] + [pad_contraction(c, 5) for c in pure[3]]
contractionsPureLower[6] = [pad_contraction(c, 6) for c in pure[4]] + [pad_contraction(c, 6) for c in pure[2]] + [pad_contraction(c, 6) for c in contractionsPureLower[2]]

contractionsMixedHom = [[] for _ in range(7)]
contractionsMixedHom[3] = [ ((1, 20, 20), (2, 21, 21), (3, 22, 22), (1, 2, 3)), ((1, 23, 23), (2, 24, 24), (1, 3, 4), (2, 3, 4)) ]
contractionsMixedHom[4] = [ ((1, 2, 20, 20), (3, 4, 21, 21), (1, 2, 3, 4)), ((1, 2, 22, 22), (2, 3, 4, 5), (1, 3, 4, 5)), ((1, 2, 23, 23), (2, 3, 24, 24), (4, 5, 25, 25), (1, 3, 4, 5)) ]
contractionsMixedHom[5] = [ ((1, 20, 20, 21, 21), (2, 22, 22, 23, 23), (3, 24, 24, 25, 25), (1, 2, 3, 26, 26)), ((1, 27, 27, 28, 28), (2, 29, 29, 30, 30), (1, 3, 4, 31, 31), (2, 3, 4, 32, 32)), ((1,33,33,34,34),(2,35,35,36,36),(3,37,37,38,38),(4,39,39,40,40),(5,41,41,42,42),(1, 2, 3,4,5)), ((1, 37, 37, 38, 38), (2, 39, 39, 40, 40), (1, 3, 4, 5, 6), (2, 3, 4, 5, 6)), ((1, 2, 3, 43, 43), (2, 3, 4, 44, 44), (5, 6, 7, 45, 45), (1, 4, 5, 6, 7)), ((1, 2, 3, 46, 46), (2, 3, 4, 47, 47), (1, 5, 6, 7, 8), (4, 5, 6, 7, 8)), ((1, 2, 3, 48, 48), (3, 4, 5, 49, 49), (1, 2, 6, 7, 8), (4, 5, 6, 7, 8)) ]
contractionsMixedHom[6] = [ ((1, 2, 20, 20, 21, 21), (3, 4, 22, 22, 23, 23), (1, 2, 3, 4, 24, 24)), ((1, 2, 25, 25, 26, 26), (2, 3, 4, 5, 27, 27), (1, 3, 4, 5, 28, 28)), ((1, 2, 29, 29, 30, 30), (2, 3, 31, 31, 32, 32), (4, 5, 33, 33, 34, 34), (1, 3, 4, 5, 35, 35)), ((1, 2, 36, 36, 37, 37), (2, 3, 4, 5, 6, 7), (1, 3, 4, 5, 6, 7)), ((1, 2, 38, 38, 39, 39), (3, 4, 40, 40, 41, 41), (5, 6, 42, 42, 43, 43), (1, 2, 3, 4, 5, 6)), ((1, 2, 44, 44, 45, 45), (2, 3, 46, 46, 47, 47), (1, 4, 5, 6, 7, 8), (3, 4, 5, 6, 7, 8)), ((1, 2, 3, 4, 48, 48), (4, 5, 6, 7, 49, 49), (1, 2, 3, 5, 6, 7)), ((1, 2, 3, 4, 50, 50), (3, 4, 5, 6, 7, 8), (1, 2, 5, 6, 7, 8)), ((1, 2, 3, 4, 51, 51), (2, 3, 4, 5, 19, 19), (6, 7, 8, 9, 0, 0), (1, 5, 6, 7, 8, 9)) ]
for i in range(1,7): 
    _ = test_independence(pure[i] + contractionsPureLower[i] + contractionsMixedHom[i], spherical=False)
print(contractionsMixedHom[5])

In [ ]:
# draw same rank different order mixed invariants 1_1 3_1, 1_1 5_1, 3_1 5_1
contractionsSim = [((1,),(1,20,20))]
i = 1
j = 3
for k in range(i, 7, +2):
    for l in range(j, 7, +2):
        print(i,k,j-2,l)
        index = 0
        for c0 in contractionsSim:
            rank_map = {i: k, j: l}
            if k<=l:
                print(rank_map, c0)
                c = pad_mixed_contraction(c0, rank_map)
                if is_mixed(c):
                    print(c)
                    g = graph_from_contraction(c)
                    fig,ax = plt.subplots(1,1,figsize=(3,3))
                    draw_graph(g)
                    # plt.show()
                    plt.savefig(f'{PLOT_DIRECTORY}/mixed^{k}_{i}^{l}_{j-2}({index}).png')  
                    plt.show()
                    plt.close()
                    index = index + 1

In [ ]:
# draw same rank different order mixed invariants 2_2 4_2, 2_ 26_2, 4_2 6_2, 4_4 6_4
contractionsSim = [((1,2),(1,2,0,0)), ((1,2),(2,3),(1,3,0,0)), ((1,2),(2,3,0,0),(1,3,9,9))]
i0=2
j0=4
for i in range(2, 7, +2):
    for j in range(4, 7, +2):
        if i<=j:
            for k in range(i, 7, +2):
                for l in range(j, 7, +2):
                    print(i,k,j-2,l)
                    index = 0
                    for c0 in contractionsSim:
                        rank_map = {i0: k, j0: l}
                        if k<=l:
                            print(rank_map, c0)
                            c = pad_mixed_contraction(c0, rank_map)
                            if is_mixed(c):
                                print(c)
                                g = graph_from_contraction(c)
                                fig,ax = plt.subplots(1,1,figsize=(3,3))
                                draw_graph(g)
                                plt.savefig(f'{PLOT_DIRECTORY}/mixed^{k}_{i}^{l}_{j-2}({index}).png')  
                                plt.show()
                                plt.close()
                                index = index + 1

In [ ]:
# draw same rank different order mixed invariants 3_3 5_3
contractionsSim = [((1,2,3),(1,2,3,20,20)), ((1,2,3),  (1,2,4),  (3,5,6),  (4,5,6,23,23)), ((1,2,3),  (1,2,4),  (3,5,6,22,22),  (4,5,6,23,23))]
i=3
j=5
for c in contractionsSim:
    print(c)
    g = graph_from_contraction(c)
    fig,ax = plt.subplots(1,1,figsize=(3,3))
    draw_graph(g)
    # plt.show()
    plt.savefig(f'{PLOT_DIRECTORY}/mixed^{k}_{i}^{l}_{j}({index}).png')  
    plt.show()
    plt.close()
    index = index + 1

In [ ]:
# test same order independence 35_3: 3
contractionsSim = [((1,2,3),(1,2,3,20,20)), ((1,2,3),  (1,2,4),  (3,5,6),  (4,5,6,23,23)), ((1,2,3),  (1,2,4),  (3,5,6,22,22),  (4,5,6,23,23)), ((1,2,3),  (1,2,4,21,21),  (3,5,6,22,22),  (4,5,6,23,23))]
i=3
j=5
contractions = pure[i] + pure[j] + contractionsPureLower[i] + contractionsPureLower[j] + contractionsMixedHom[i] + contractionsMixedHom[j] + contractionsSim
grads = test_independence(contractions, spherical=False)



In [ ]:
# test same order independence 2_2 6_2: 3
contractionsSim = [((1,2),(1,2,0,0,9,9)), ((1,2),(2,3),(1,3,0,0,9,9)), ((1,2),(2,3,0,0,8,8),(1,3,9,9,7,7))]
i=2
j=6
contractions = pure[i] + pure[j] + contractionsPureLower[i] + contractionsPureLower[j] + contractionsMixedHom[i] + contractionsMixedHom[j] + contractionsSim
grads = test_independence(contractions, spherical=False)
# print(pure[i])
# print(pure[j])
# print(contractionsMixedHom[i])
# print(contractionsMixedHom[j])
# print(contractionsPureLower[i])
# print(contractionsPureLower[j])
# print(contractionsSim)

In [ ]:
# test same order independence 4_4 6_4: 3
contractionsSim = [((1, 2, 3, 4),  (1, 2, 3, 4,0,0)), ((1, 2, 3, 4),  (3, 4, 5, 6),  (1, 2, 5, 6,9,9)), ((1, 2, 3, 4),  (3, 4, 5, 6,0,0),  (1, 2, 5, 6,9,9)), ((1, 2, 3, 4),(2, 3, 4, 5),(1, 6, 7, 8),(5, 6, 7, 8,0,0))]
i=4
j=6
contractions = pure[i] + pure[j] + contractionsPureLower[i] + contractionsPureLower[j] + contractionsMixedHom[i] + contractionsMixedHom[j] + contractionsSim
grads = test_independence(contractions, spherical=False)

In [ ]:
# test same order independence  4_2 6_2: 3
contractionsSim = [((1,2,8,8),(1,2,0,0,9,9)), ((1,2,8,8),(2,3,7,7),(1,3,0,0,9,9)), ((1,2,6,6),(2,3,0,0,8,8),(1,3,9,9,7,7))]
i=4
j=6
contractions = pure[i] + pure[j] + contractionsMixedHom[i] + contractionsMixedHom[j] + contractionsSim
grads = test_independence(contractions, spherical=False)

In [ ]:
# bujack
contractions = [((1,1,2),(2,3,4),(3,4,5),(5,6,6)), ((1,1,2),(2,3,4),(3,4,5),(5,6,7),(6,7,8),(8,9,9))]
for index, c in enumerate(contractions):
    g = graph_from_contraction(c)
    fig,ax = plt.subplots(1,1,figsize=(3,3))
    draw_graph(g)
    plt.savefig(f'{PLOT_DIRECTORY}/bujackNeu{index}.png')
    plt.show()
    plt.close()

In [ ]:
# langbein
contractions = [((1, 2, 3), (1, 2, 3)), ((1, 1, 2), (2, 3, 3)), ((1, 2, 3), (4,5,6), (1, 5, 6), (2,3,4)), ((1, 2, 3), (1,4, 5), (2, 5, 6), (3, 4, 6)), ((1, 2, 3), (1, 4, 5), (2, 4, 5), (3, 6, 6)), ((1, 2, 3), (1, 4, 4), (2, 5, 5), (3, 6, 6)) ]
for index, c in enumerate(contractions):
    g = graph_from_contraction(c)
    fig,ax = plt.subplots(1,1,figsize=(3,3))
    draw_graph(g)
    plt.savefig(f'{PLOT_DIRECTORY}/langbein{index}.png')
    plt.show()
    plt.close()
